In [1]:
import discord
import os
from dotenv import load_dotenv

from api_keys import *
from command_handling import *

from DESTINY import basics
from DESTINY import manifestation
from DESTINY import hbfunc

from selenium import webdriver

import bs4
import urllib

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

import json
import requests
import shutil

import numpy as np

In [2]:
all_data = manifestation.get_all_data(regen=False)

Not regenerating manifest


In [3]:
weapon_dict = manifestation.get_weapon_dict(all_data)

In [4]:
perk_dict = manifestation.get_perk_dict(all_data)

In [5]:
for ep in all_data.keys():
    print(ep)

DestinyPlaceDefinition
DestinyActivityDefinition
DestinyActivityTypeDefinition
DestinyClassDefinition
DestinyGenderDefinition
DestinyInventoryBucketDefinition
DestinyRaceDefinition
DestinyTalentGridDefinition
DestinyUnlockDefinition
DestinyMaterialRequirementSetDefinition
DestinySandboxPerkDefinition
DestinyStatGroupDefinition
DestinyFactionDefinition
DestinyVendorGroupDefinition
DestinyRewardSourceDefinition
DestinyItemCategoryDefinition
DestinyDamageTypeDefinition
DestinyActivityModeDefinition
DestinyMedalTierDefinition
DestinyAchievementDefinition
DestinyActivityGraphDefinition
DestinyBondDefinition
DestinyCollectibleDefinition
DestinyDestinationDefinition
DestinyEquipmentSlotDefinition
DestinyStatDefinition
DestinyInventoryItemDefinition
DestinyItemTierTypeDefinition
DestinyLocationDefinition
DestinyLoreDefinition
DestinyMetricDefinition
DestinyObjectiveDefinition
DestinyPlugSetDefinition
DestinyPowerCapDefinition
DestinyPresentationNodeDefinition
DestinyProgressionDefinition
Desti

In [6]:
def find_id(search_id):
    for key in all_data.keys():
        if search_id in all_data[key].keys():
            print(key, "\n", json.dumps(all_data[key][search_id], indent=4))

###  Statmax/statmin command:
similar to sametype, but find the one with the highest/lowest stat in a given category

In [7]:
import difflib

Pseudocode for  statmin/statmax:  
ah idk if I wanna bother with types and archetypes ehhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh


In [8]:
def remove_outer_spaces(s:str):
    """
    Remove leading or trailing spaces from string s and  return
    """
    mid_s = ""
    
    # handling leading
    leading=True
    for c in s:
        if leading is True:
            if c == " ":
                continue
            else:
                leading=False
                mid_s+=c
        
        else:
            mid_s += c
            
    final_s_r = ""
    
    # handling trailing
    n = len(mid_s)
    trailing=True
    for i in range(len(mid_s)):
        i_r = n - i - 1
        
        c_r  = mid_s[i_r]
        if trailing is True:
            if c_r == " ":
                continue
            else:
                final_s_r+=c_r
                trailing=False
                
        else:
            final_s_r+=c_r
    
    # reverse
    final_s = ""
    for i in range(len(final_s_r)):
        i_r = len(final_s_r) - i - 1
        c_r = final_s_r[i_r]
        final_s += c_r
    
    
    return final_s


remove_outer_spaces("    ab cde ")

'ab cde'

In [9]:
def statorder(message, args, client, all_data, weapon_dict):
    # need to also specify desired stat, which would come from args...
    weap, stat = " ".join(args).split("&")
    weap = remove_outer_spaces(weap)
    stat = remove_outer_spaces(stat)
    print(weap, stat)
    sametype_string = hbfunc.sametype(message, args, client, all_data, weapon_dict)
    
    for i, line in enumerate(sametype_string.split("\n")):
        # skip the first one
        if i ==0:
            continue
            
        # skip line breaks
        if len(line)==0:
            continue
            
        # if you reach these tiers, end the loop
        if line in ["**RARE**", "**UNCOMMON**",  "**COMMON**"]:
            break
        
        # if it's a title or 
        elif line in ["**EXOTIC**", "**LEGENDARY**"]:
            continue
            
        else:
            # this is an Exotic or Legendary weapon name
            print(line)
            print(hbfunc.weapstat(message, line.split(" "), client, all_data, weapon_dict))
    

statorder(None, ["The", "Palindrome", "&", "zoom"], None, all_data, weapon_dict)

The Palindrome zoom
Ace of Spades
Name: Ace of Spades
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 78
Stability: 46
Magazine: 13
Reload Speed: 48
Handling: 46
Aim Assistance: 70
Recoil Direction: 100

Hawkmoon
Name: Hawkmoon
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 78
Range: 57
Stability: 67
Magazine: 8
Reload Speed: 63
Handling: 74
Aim Assistance: 93
Recoil Direction: 96

Lumina
Name: Lumina
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 44
Stability: 46
Magazine: 13
Reload Speed: 65
Handling: 74
Aim Assistance: 88
Recoil Direction: 95

Thorn
Name: Thorn
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 41
Stability: 60
Magazine: 11
Reload Speed: 46
Handling: 68
Aim Assistance: 85
Recoil Direction: 100

Ancient Gospel
Name: Ancient Gospel
Type: Legendary Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 47
Stability: 62
Magazine: 10
Reload Speed: 49
Handling: 51
Aim A

In [10]:
print(hbfunc.weapstat(None, ["Fatebringer","(Timelost)"], None, all_data, weapon_dict))

Name: Fatebringer (Timelost)
Type: Legendary Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 51
Stability: 61
Magazine: 10
Reload Speed: 51
Handling: 60
Aim Assistance: 84
Recoil Direction: 98



## Perk combo function
$combo Demolitionist & Adrenaline Junkie --> return weapons that can roll both *in different slots*

In [11]:
from DESTINY.hbfunc import get_closest_gun
#from DESTINY.hbfunc import rolls
import helpers

In [12]:
import inspect  # get function source code
#lines = inspect.getsource(rolls)  
#print(lines)

In [13]:
def rolls_(message, args, client, all_data, weapon_dict, retstr=True):
    """
    get all possible perk rolls for a given weapon
    """

    #weapon = " ".join(args)
    weapon = get_closest_gun(message, args, client, all_data, weapon_dict)[0]

    repeats = 1
    rolls = {}

    # some weapons (raid, exotic, etc) have a different index for rolls
    # breaks for some other weapons...
    
    if len(weapon_dict[weapon]) == 3:
        idx = 1
    elif len(weapon_dict[weapon]) == 4:
        idx = 2
    else:
        idx = 0
    

    weaptype = weapon_dict[weapon][idx]['itemTypeAndTierDisplayName']
    
    print(weaptype)
    
    # for socket (usually barrels, mags, perk1, perk2, etc)
    for socket in weapon_dict[weapon][idx]['sockets']['socketEntries']:
        # if random rolls
        if "randomizedPlugSetHash" in socket.keys():
            randhash = socket['randomizedPlugSetHash']
            sockethash = socket['socketTypeHash']
            socketname = all_data['DestinySocketTypeDefinition'][sockethash]['plugWhitelist'][0]['categoryIdentifier']
        
            # for every perk that goes in that socket for this weapon
            perks = set()
            for perk in all_data['DestinyPlugSetDefinition'][randhash]['reusablePlugItems']:
                itemhash = perk['plugItemHash']
                perks.add(all_data['DestinyInventoryItemDefinition'][itemhash]['displayProperties']['name'])
                
            # save list of perks for each socket name
            if socketname == "frames":
                socketname = "Main Perk " + str(repeats)
                repeats += 1
            else:
                socketname = socketname[0].upper() + socketname[1:]
                
            rolls[socketname] = list(perks)


    if retstr is True:
        return(rolls_format(rolls, weapon, weaptype))

    return rolls # can specify retstr = False in parameters if you just want the dict back.

In [14]:
rolls_(None, ["Finite", "Impactor"], None, all_data, weapon_dict, retstr=False) 

Legendary Hand Cannon


{'Scopes': ['HitMark HCS',
  'TrueSight HCS',
  'Sureshot HCS',
  'Fastdraw HCS',
  'SteadyHand HCS',
  'Crossfire HCS'],
 'Magazines': ['Ricochet Rounds',
  'High-Caliber Rounds',
  'Light Mag',
  'Alloy Magazine',
  'Flared Magwell',
  'Armor-Piercing Rounds',
  'Appended Mag',
  'Extended Mag'],
 'Main Perk 1': ['No Distractions',
  'Heating Up',
  'Under Pressure',
  'Firmly Planted',
  'Killing Wind',
  'Slideways'],
 'Main Perk 2': ['Elemental Capacitor',
  'Iron Grip',
  'Eye of the Storm',
  'One for All',
  'Iron Reach',
  'Multikill Clip']}

In [15]:
rolls_(None, ["Age-Old" ,"Bond"], None, all_data, weapon_dict, retstr=False) 

Legendary Auto Rifle


{'Barrels': ['Smallbore',
  'Chambered Compensator',
  'Arrowhead Brake',
  'Extended Barrel',
  'Full Bore',
  'Corkscrew Rifling',
  'Polygonal Rifling',
  'Hammer-Forged Rifling',
  'Fluted Barrel'],
 'Magazines': ['Ricochet Rounds',
  'High-Caliber Rounds',
  'Light Mag',
  'Alloy Magazine',
  'Armor-Piercing Rounds',
  'Extended Mag'],
 'Main Perk 1': ['Tap the Trigger',
  'Dynamic Sway Reduction',
  'Zen Moment',
  'Triple Tap'],
 'Main Perk 2': ['Kill Clip',
  'High-Impact Reserves',
  'Dragonfly',
  'Ambitious Assassin']}

In [16]:
rolls_(None, ["Heritage"], None, all_data, weapon_dict, retstr=False) 

Legendary Shotgun


{'Barrels': ['Smallbore',
  'Chambered Compensator',
  'Arrowhead Brake',
  'Extended Barrel',
  'Full Bore',
  'Corkscrew Rifling',
  'Polygonal Rifling',
  'Hammer-Forged Rifling',
  'Fluted Barrel'],
 'Magazines': ['Light Mag',
  'Steady Rounds',
  'Tactical Mag',
  'Appended Mag',
  'Assault Mag',
  'Extended Mag',
  'Accurized Rounds'],
 'Main Perk 1': ['Dual Loader',
  'Slideshot',
  'Outlaw',
  'Auto-Loading Holster',
  'Hip-Fire Grip',
  'Reconstruction'],
 'Main Perk 2': ['Unrelenting',
  'Snapshot Sights',
  'Moving Target',
  'Recombination',
  'Killing Wind',
  'Thresh']}

In [17]:
lens = []
for w in weapon_dict.keys():
    ln = len(weapon_dict[w])
    lens.append(ln)
    
max(lens)

3

In [18]:
import os

In [19]:
os.system("echo hello")

hello


0

In [38]:
os.system("git log > gitlog.txt")

0

In [39]:
with open('gitlog.txt', 'r') as file:
    data = "\n"+file.read()

In [53]:
for commit in data.split("\ncommit "): # not perfect but
    if len(commit) > 0:
        #print(commit)
        print(commit.split("\n")[0])
        if "https://github.com/maxshinnerl" not in "".join(commit.split("\n")[4:]):
            print("".join(commit.split("\n")[4:]).replace("    ", " "))

d6ba44ee3c906374a0f87e90825a0476ef4654a3
 Two line commit for testing
e8e6d2339fa89220ff89378ea821ec231ba24ce3
 patch notes will now display in random instead of dev channel
ca1c17f4ef1d68442093940b5e9151aef6350fee
 added auto discord patch notes on HBOT restart
46e7c5ffcd6de323e1898e7e5b38db2c5ca0ae83
 removed bullying
431e764a407f5b665fe21e19c69e0f3184dbca92
 tofu bozo
0fc311bbe8ca921eae639f8831033bb2b9d9468c
 added  command
236a783b4322604dd78a33f13f7211179d190dcd
 fixed rpm and charge time in compare
8df9329a9ce851a25a270fa5a0c4be715eef01bd
 removed ROF from
ae5c42ba8ca4b62ace05a3bc59e7a2ae4a93d9fc
 added please wait message for long funcs
acbb987aaf8f3c18e27f73f47c44150a46d9629d
 comment
9c5a47c96c1ce6545e7e6e31d646c267e6aee3be
 typo
ff5f2c03563a318d110cc612d06900a64a541494
 notebook updates
09d22692a2c6cc2d9b98d6a51f980e17cb09d709
 rolls fix again
3f0b11b3a6c5b57e2aef01be15d8f27dbe911c90
 fixed 5/5 response
e3687e41f73c007ba9fe973a07cc26c97182a5e0
 added hbot birthday stuff
0983b